In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

import math

import torchtext

import datasets

from tqdm import tqdm

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from datasets import load_dataset
nltk.download('stopwords')
nltk.download('punkt')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(0)

[nltk_data] Downloading package stopwords to /home/duy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/duy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
from datasets import load_dataset
dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"].filter(lambda example, idx: example['language'] == 'arabic', with_indices=True)
valid_set = dataset["validation"].filter(lambda example, idx: example['language'] == 'arabic', with_indices=True)

In [43]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
        num_rows: 116067
    })
    validation: Dataset({
        features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
        num_rows: 13325
    })
})


In [18]:
# train_set = train_set['document_plaintext']
# valid_set = valid_set['document_plaintext']

tokenizer = torchtext.data.utils.get_tokenizer(wordpunct_tokenize)
tokenize_data = lambda example, tokenizer: {'tokens': tokenizer(example['document_plaintext'])}  
tokenized_dataset = train_set.map(tokenize_data, remove_columns=['question_text', 'document_title', 
                                                                 'language', 'annotations', 
                                                                 'document_plaintext', 'document_url'], 
fn_kwargs={'tokenizer': tokenizer})

tokenized_valid_dataset = valid_set.map(tokenize_data, remove_columns=['question_text', 'document_title', 
                                                                 'language', 'annotations', 
                                                                 'document_plaintext', 'document_url'], 
fn_kwargs={'tokenizer': tokenizer})

Map: 100%|████████████████████████████████████████████| 1902/1902 [00:00<00:00, 20022.50 examples/s]


['و', 'ُ', 'لد', 'نابليون', 'في', 'جزيرة', 'كورسيكا', 'لأبوين', 'ينتميان', 'لطبقة', 'أرستقراطية', 'تعود', 'بجذورها', 'إلى', 'إحدى', 'عائلات', 'إيطاليا', 'القديمة', 'النبيلة', '.', 'ألحقه', 'والده', '"', 'كارلو', 'بونابرت', '"،', 'المعروف', 'عند', 'الفرنسيين', 'باسم', '"', 'شارل', 'بونابرت', '"', 'بمدرسة', 'بريان', 'العسكرية', '.', 'ثم', 'التحق', 'بعد', 'ذلك', 'بمدرسة', 'سان', 'سير', 'العسكرية', 'الشهيرة', '،', 'وفي', 'المدرستين', 'أظهر', 'تفوق', 'ً', 'ا', 'باهر', 'ً', 'ا', 'على', 'رفاقه', '،', 'ليس', 'فقط', 'في', 'العلوم', 'العسكرية', 'وإنما', 'أيض', 'ً', 'ا', 'في', 'الآداب', 'والتاريخ', 'والجغرافيا', '.', 'وخلال', 'دراسته', 'اطلع', 'على', 'روائع', 'كت', 'ّ', 'اب', 'القرن', 'الثامن', 'عشر', 'في', 'فرنسا', 'وجل', 'ّ', 'هم', '،', 'حيث', 'كانوا', 'من', 'أصحاب', 'ودعاة', 'المبادئ', 'الحرة', '.', 'فقد', 'عرف', 'عن', 'كثب', 'مؤلفات', 'فولتير', 'ومونتسكيو', 'وروسو', '،', 'الذي', 'كان', 'أكثرهم', 'أثر', 'ً', 'ا', 'في', 'تفكير', 'الضابط', 'الشاب', '.']


In [20]:
vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_dataset['tokens'], 
min_freq=3)
vocab.insert_token('<unk>', 0)           
vocab.insert_token('<eos>', 1)            
vocab.set_default_index(vocab['<unk>'])   

67006
['<unk>', '<eos>', '،', 'في', '.', 'من', 'على', ':', 'ً', ']']


In [2]:
def get_data(dataset, vocab, batch_size):
    data = []                                                   
    for example in dataset:
        if example['tokens']:                                      
            tokens = example['tokens'].append('<eos>')             
            tokens = [vocab[token] for token in example['tokens']] 
            data.extend(tokens)                                    
    data = torch.LongTensor(data)                                 
    num_batches = data.shape[0] // batch_size 
    data = data[:num_batches * batch_size]                       
    data = data.view(batch_size, num_batches)          
    return data

In [22]:
batch_size = 128
train_data = get_data(tokenized_dataset, vocab, batch_size)
valid_data = get_data(tokenized_valid_dataset, vocab, batch_size)

In [3]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, 
                tie_weights):
                
        super().__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, 
                    dropout=dropout_rate, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        
        if tie_weights:
            assert embedding_dim == hidden_dim, 'cannot tie, check dims'
            self.embedding.weight = self.fc.weight
        self.init_weights()

    def forward(self, src, hidden):
        embedding = self.dropout(self.embedding(src))
        output, hidden = self.lstm(embedding, hidden)          
        output = self.dropout(output) 
        prediction = self.fc(output)
        return prediction, hidden

    def init_weights(self):
        init_range_emb = 0.1
        init_range_other = 1/math.sqrt(self.hidden_dim)
        self.embedding.weight.data.uniform_(-init_range_emb, init_range_emb)
        self.fc.weight.data.uniform_(-init_range_other, init_range_other)
        self.fc.bias.data.zero_()
        for i in range(self.num_layers):
            self.lstm.all_weights[i][0] = torch.FloatTensor(self.embedding_dim,
                    self.hidden_dim).uniform_(-init_range_other, init_range_other) 
            self.lstm.all_weights[i][1] = torch.FloatTensor(self.hidden_dim, 
                    self.hidden_dim).uniform_(-init_range_other, init_range_other) 

    def init_hidden(self, batch_size, device):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        return hidden, cell

    def detach_hidden(self, hidden):
        hidden, cell = hidden
        hidden = hidden.detach()
        cell = cell.detach()
        return hidden, cell

In [4]:
def get_batch(data, seq_len, num_batches, idx):
    src = data[:, idx:idx+seq_len]                   
    target = data[:, idx+1:idx+seq_len+1]             
    return src, target

In [5]:
def train(model, data, optimizer, criterion, batch_size, seq_len, clip, device):
    
    epoch_loss = 0
    model.train()
    # drop all batches that are not a multiple of seq_len
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)
    
    for idx in tqdm(range(0, num_batches - 1, seq_len), desc='Training: ',leave=False):  # The last batch can't be a src
        optimizer.zero_grad()
        hidden = model.detach_hidden(hidden)

        src, target = get_batch(data, seq_len, num_batches, idx)
        src, target = src.to(device), target.to(device)
        batch_size = src.shape[0]
        prediction, hidden = model(src, hidden)               

        prediction = prediction.reshape(batch_size * seq_len, -1)   
        target = target.reshape(-1)
        loss = criterion(prediction, target)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item() * seq_len
    return epoch_loss / num_batches

In [6]:
def evaluate(model, data, criterion, batch_size, seq_len, device):

    epoch_loss = 0
    model.eval()
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)

    with torch.no_grad():
        for idx in range(0, num_batches - 1, seq_len):
            hidden = model.detach_hidden(hidden)
            src, target = get_batch(data, seq_len, num_batches, idx)
            src, target = src.to(device), target.to(device)
            batch_size= src.shape[0]

            prediction, hidden = model(src, hidden)
            prediction = prediction.reshape(batch_size * seq_len, -1)
            target = target.reshape(-1)

            loss = criterion(prediction, target)
            epoch_loss += loss.item() * seq_len
    return epoch_loss / num_batches

In [30]:
vocab_size = len(vocab)
embedding_dim = 400             # 400 in the paper
hidden_dim = 400                # 1150 in the paper
num_layers = 2                   # 3 in the paper
dropout_rate = 0.65              
tie_weights = True                  
lr = 1e-3   

In [31]:
model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

The model has 29,435,806 trainable parameters


In [7]:
def train_model(model, train_data, valid_data, fname):
    n_epochs = 50
    seq_len = 50
    clip = 0.25
    saved = False
    
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)
    
    if saved:
        model.load_state_dict(torch.load('best-val-lstm_lm.pt',  map_location=device))
        test_loss = evaluate(model, test_data, criterion, batch_size, seq_len, device)
        print(f'Test Perplexity: {math.exp(test_loss):.3f}')
    else:
        best_valid_loss = float('inf')
    
        for epoch in range(n_epochs):
            train_loss = train(model, train_data, optimizer, criterion, 
                        batch_size, seq_len, clip, device)
            valid_loss = evaluate(model, valid_data, criterion, batch_size, 
                        seq_len, device)
            
            lr_scheduler.step(train_loss)
    
            if train_loss < best_valid_loss:
                best_valid_loss = valid_loss
                torch.save(model.state_dict(), fname)
    
            print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
            print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

In [10]:
dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"].filter(lambda example, idx: example['language'] == 'arabic', with_indices=True)
valid_set = dataset["validation"].filter(lambda example, idx: example['language'] == 'arabic', with_indices=True)

f = lambda x: wordpunct_tokenize(x.lower())
tokenizer = torchtext.data.utils.get_tokenizer(f)
tokenize_data = lambda example, tokenizer: {'tokens': tokenizer(example['document_plaintext'])}  
tokenized_dataset = train_set.map(tokenize_data, remove_columns=['question_text', 'document_title', 
                                                                 'language', 'annotations', 
                                                                 'document_plaintext', 'document_url'], 
fn_kwargs={'tokenizer': tokenizer})

tokenized_valid_dataset = valid_set.map(tokenize_data, remove_columns=['question_text', 'document_title', 
                                                                 'language', 'annotations', 
                                                                 'document_plaintext', 'document_url'], 
fn_kwargs={'tokenizer': tokenizer})

vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_dataset['tokens'], 
min_freq=3)
vocab.insert_token('<unk>', 0)           
vocab.insert_token('<eos>', 1)            
vocab.set_default_index(vocab['<unk>']) 

batch_size = 128
train_data = get_data(tokenized_dataset, vocab, batch_size)
valid_data = get_data(tokenized_valid_dataset, vocab, batch_size)

vocab_size = len(vocab)
embedding_dim = 400             # 400 in the paper
hidden_dim = 400                # 1150 in the paper
num_layers = 2                   # 3 in the paper
dropout_rate = 0.65              
tie_weights = True                  
lr = 1e-3   

model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

fname = 'arabic_document.pt'
train_model(model, train_data, valid_data, fname)

The model has 29,444,227 trainable parameters


	Train Perplexity: 3245.628
	Valid Perplexity: 1344.297


	Train Perplexity: 1469.439
	Valid Perplexity: 797.537


	Train Perplexity: 1002.867
	Valid Perplexity: 594.356


	Train Perplexity: 775.336
	Valid Perplexity: 488.170


...

	Train Perplexity: 158.165
	Valid Perplexity: 206.362


	Train Perplexity: 156.972
	Valid Perplexity: 206.454


	Train Perplexity: 155.709
	Valid Perplexity: 205.702


	Train Perplexity: 154.802
	Valid Perplexity: 204.902


	Train Perplexity: 153.725
	Valid Perplexity: 204.767


In [11]:
dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"].filter(lambda example, idx: example['language'] == 'arabic', with_indices=True)
valid_set = dataset["validation"].filter(lambda example, idx: example['language'] == 'arabic', with_indices=True)

f = lambda x: wordpunct_tokenize(x.lower())
tokenizer = torchtext.data.utils.get_tokenizer(f)
tokenize_data = lambda example, tokenizer: {'tokens': tokenizer(example['question_text'])}  
tokenized_dataset = train_set.map(tokenize_data, remove_columns=['question_text', 'document_title', 
                                                                 'language', 'annotations', 
                                                                 'document_plaintext', 'document_url'], 
fn_kwargs={'tokenizer': tokenizer})

tokenized_valid_dataset = valid_set.map(tokenize_data, remove_columns=['question_text', 'document_title', 
                                                                 'language', 'annotations', 
                                                                 'document_plaintext', 'document_url'], 
fn_kwargs={'tokenizer': tokenizer})

vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_dataset['tokens'], 
min_freq=3)
vocab.insert_token('<unk>', 0)           
vocab.insert_token('<eos>', 1)            
vocab.set_default_index(vocab['<unk>']) 

batch_size = 128
train_data = get_data(tokenized_dataset, vocab, batch_size)
valid_data = get_data(tokenized_valid_dataset, vocab, batch_size)

vocab_size = len(vocab)
embedding_dim = 400             # 400 in the paper
hidden_dim = 400                # 1150 in the paper
num_layers = 2                   # 3 in the paper
dropout_rate = 0.65              
tie_weights = True                  
lr = 1e-3   

model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

fname = 'arabic_question.pt'
train_model(model, train_data, valid_data, fname)

Map: 100%|████████████████████████████████████████████| 1902/1902 [00:00<00:00, 28779.93 examples/s]


The model has 5,025,733 trainable parameters


	Train Perplexity: 542.551
	Valid Perplexity: 137.617


	Train Perplexity: 268.661
	Valid Perplexity: 131.977


	Train Perplexity: 227.531
	Valid Perplexity: 92.652


...

	Train Perplexity: 17.713
	Valid Perplexity: 19.472


	Train Perplexity: 17.464
	Valid Perplexity: 19.440


	Train Perplexity: 17.186
	Valid Perplexity: 19.409


	Train Perplexity: 17.031
	Valid Perplexity: 19.380


	Train Perplexity: 16.765
	Valid Perplexity: 19.456


In [12]:
dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"].filter(lambda example, idx: example['language'] == 'indonesian', with_indices=True)
valid_set = dataset["validation"].filter(lambda example, idx: example['language'] == 'indonesian', with_indices=True)

f = lambda x: wordpunct_tokenize(x.lower())
tokenizer = torchtext.data.utils.get_tokenizer(f)
tokenize_data = lambda example, tokenizer: {'tokens': tokenizer(example['question_text'])}  
tokenized_dataset = train_set.map(tokenize_data, remove_columns=['question_text', 'document_title', 
                                                                 'language', 'annotations', 
                                                                 'document_plaintext', 'document_url'], 
fn_kwargs={'tokenizer': tokenizer})

tokenized_valid_dataset = valid_set.map(tokenize_data, remove_columns=['question_text', 'document_title', 
                                                                 'language', 'annotations', 
                                                                 'document_plaintext', 'document_url'], 
fn_kwargs={'tokenizer': tokenizer})

vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_dataset['tokens'], 
min_freq=3)
vocab.insert_token('<unk>', 0)           
vocab.insert_token('<eos>', 1)            
vocab.set_default_index(vocab['<unk>']) 

batch_size = 128
train_data = get_data(tokenized_dataset, vocab, batch_size)
valid_data = get_data(tokenized_valid_dataset, vocab, batch_size)

vocab_size = len(vocab)
embedding_dim = 400             # 400 in the paper
hidden_dim = 400                # 1150 in the paper
num_layers = 2                   # 3 in the paper
dropout_rate = 0.65              
tie_weights = True                  
lr = 1e-3   

model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

fname = 'indonesian_question.pt'
train_model(model, train_data, valid_data, fname)

Map: 100%|████████████████████████████████████████████| 1191/1191 [00:00<00:00, 28807.80 examples/s]


The model has 3,532,810 trainable parameters


	Train Perplexity: 616.464
	Valid Perplexity: 87.028


	Train Perplexity: 196.093
	Valid Perplexity: 88.368


	Train Perplexity: 179.784
	Valid Perplexity: 84.206


...

	Train Perplexity: 18.073
	Valid Perplexity: 15.694


	Train Perplexity: 17.786
	Valid Perplexity: 15.588


	Train Perplexity: 17.510
	Valid Perplexity: 15.525


	Train Perplexity: 17.373
	Valid Perplexity: 15.443


	Train Perplexity: 17.187
	Valid Perplexity: 15.395


In [13]:
dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"].filter(lambda example, idx: example['language'] == 'indonesian', with_indices=True)
valid_set = dataset["validation"].filter(lambda example, idx: example['language'] == 'indonesian', with_indices=True)

f = lambda x: wordpunct_tokenize(x.lower())
tokenizer = torchtext.data.utils.get_tokenizer(f)
tokenize_data = lambda example, tokenizer: {'tokens': tokenizer(example['document_plaintext'])}  
tokenized_dataset = train_set.map(tokenize_data, remove_columns=['question_text', 'document_title', 
                                                                 'language', 'annotations', 
                                                                 'document_plaintext', 'document_url'], 
fn_kwargs={'tokenizer': tokenizer})

tokenized_valid_dataset = valid_set.map(tokenize_data, remove_columns=['question_text', 'document_title', 
                                                                 'language', 'annotations', 
                                                                 'document_plaintext', 'document_url'], 
fn_kwargs={'tokenizer': tokenizer})

vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_dataset['tokens'], 
min_freq=3)
vocab.insert_token('<unk>', 0)           
vocab.insert_token('<eos>', 1)            
vocab.set_default_index(vocab['<unk>']) 

batch_size = 128
train_data = get_data(tokenized_dataset, vocab, batch_size)
valid_data = get_data(tokenized_valid_dataset, vocab, batch_size)

vocab_size = len(vocab)
embedding_dim = 400             # 400 in the paper
hidden_dim = 400                # 1150 in the paper
num_layers = 2                   # 3 in the paper
dropout_rate = 0.65              
tie_weights = True                  
lr = 1e-3   

model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

fname = 'indonesian_document.pt'
train_model(model, train_data, valid_data, fname)

Map: 100%|████████████████████████████████████████████| 1191/1191 [00:00<00:00, 21047.07 examples/s]


The model has 11,473,412 trainable parameters


	Train Perplexity: 1728.563
	Valid Perplexity: 839.641


	Train Perplexity: 1078.626
	Valid Perplexity: 632.249


	Train Perplexity: 819.564
	Valid Perplexity: 452.362


...

	Train Perplexity: 94.393
	Valid Perplexity: 132.168


	Train Perplexity: 93.250
	Valid Perplexity: 132.836


	Train Perplexity: 92.722
	Valid Perplexity: 132.628


	Train Perplexity: 91.892
	Valid Perplexity: 131.539


	Train Perplexity: 90.976
	Valid Perplexity: 131.655


In [14]:
dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"].filter(lambda example, idx: example['language'] == 'bengali', with_indices=True)
valid_set = dataset["validation"].filter(lambda example, idx: example['language'] == 'bengali', with_indices=True)

f = lambda x: wordpunct_tokenize(x.lower())
tokenizer = torchtext.data.utils.get_tokenizer(f)
tokenize_data = lambda example, tokenizer: {'tokens': tokenizer(example['document_plaintext'])}  
tokenized_dataset = train_set.map(tokenize_data, remove_columns=['question_text', 'document_title', 
                                                                 'language', 'annotations', 
                                                                 'document_plaintext', 'document_url'], 
fn_kwargs={'tokenizer': tokenizer})

tokenized_valid_dataset = valid_set.map(tokenize_data, remove_columns=['question_text', 'document_title', 
                                                                 'language', 'annotations', 
                                                                 'document_plaintext', 'document_url'], 
fn_kwargs={'tokenizer': tokenizer})

vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_dataset['tokens'], 
min_freq=3)
vocab.insert_token('<unk>', 0)           
vocab.insert_token('<eos>', 1)            
vocab.set_default_index(vocab['<unk>']) 

batch_size = 128
train_data = get_data(tokenized_dataset, vocab, batch_size)
valid_data = get_data(tokenized_valid_dataset, vocab, batch_size)

vocab_size = len(vocab)
embedding_dim = 400             # 400 in the paper
hidden_dim = 400                # 1150 in the paper
num_layers = 2                   # 3 in the paper
dropout_rate = 0.65              
tie_weights = True                  
lr = 1e-3   

model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

fname = 'bengali_document.pt'
train_model(model, train_data, valid_data, fname)

Map: 100%|██████████████████████████████████████████████| 224/224 [00:00<00:00, 10546.85 examples/s]


The model has 4,486,388 trainable parameters


	Train Perplexity: 90.349
	Valid Perplexity: 44.236


	Train Perplexity: 41.374
	Valid Perplexity: 30.113


	Train Perplexity: 31.798
	Valid Perplexity: 24.314


...

	Train Perplexity: 10.597
	Valid Perplexity: 9.194


	Train Perplexity: 10.542
	Valid Perplexity: 9.180


	Train Perplexity: 10.512
	Valid Perplexity: 9.154


	Train Perplexity: 10.466
	Valid Perplexity: 9.127


	Train Perplexity: 10.427
	Valid Perplexity: 9.112


In [15]:
dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"].filter(lambda example, idx: example['language'] == 'bengali', with_indices=True)
valid_set = dataset["validation"].filter(lambda example, idx: example['language'] == 'bengali', with_indices=True)

f = lambda x: wordpunct_tokenize(x.lower())
tokenizer = torchtext.data.utils.get_tokenizer(f)
tokenize_data = lambda example, tokenizer: {'tokens': tokenizer(example['question_text'])}  
tokenized_dataset = train_set.map(tokenize_data, remove_columns=['question_text', 'document_title', 
                                                                 'language', 'annotations', 
                                                                 'document_plaintext', 'document_url'], 
fn_kwargs={'tokenizer': tokenizer})

tokenized_valid_dataset = valid_set.map(tokenize_data, remove_columns=['question_text', 'document_title', 
                                                                 'language', 'annotations', 
                                                                 'document_plaintext', 'document_url'], 
fn_kwargs={'tokenizer': tokenizer})

vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_dataset['tokens'], 
min_freq=3)
vocab.insert_token('<unk>', 0)           
vocab.insert_token('<eos>', 1)            
vocab.set_default_index(vocab['<unk>']) 

batch_size = 128
train_data = get_data(tokenized_dataset, vocab, batch_size)
valid_data = get_data(tokenized_valid_dataset, vocab, batch_size)

vocab_size = len(vocab)
embedding_dim = 400             # 400 in the paper
hidden_dim = 400                # 1150 in the paper
num_layers = 2                   # 3 in the paper
dropout_rate = 0.65              
tie_weights = True                  
lr = 1e-3   

model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

fname = 'bengali_question.pt'
train_model(model, train_data, valid_data, fname)

Map: 100%|██████████████████████████████████████████████| 224/224 [00:00<00:00, 21299.57 examples/s]


The model has 2,871,160 trainable parameters


	Train Perplexity: 110.952
	Valid Perplexity: 51.940


	Train Perplexity: 59.405
	Valid Perplexity: 49.895


	Train Perplexity: 56.430
	Valid Perplexity: 47.370


...

	Train Perplexity: 6.848
	Valid Perplexity: 7.626


	Train Perplexity: 6.720
	Valid Perplexity: 7.608


	Train Perplexity: 6.601
	Valid Perplexity: 7.518


	Train Perplexity: 6.512
	Valid Perplexity: 7.436


	Train Perplexity: 6.412
	Valid Perplexity: 7.454
